In [ ]:
!pip install wikipedia-api


## Document_download

In [ ]:
film_list = ["Dune: Part Two","Spider-Man: Far From Home","Uncharted (film)","The Great Gatsby (2013 film)","Don't Look Up","Inception","Harry Potter and the Goblet of Fire (film)","Harry Potter and the Philosopher's Stone (film)","Oppenheimer (film)","Interstellar (film)"]

In [ ]:
import wikipediaapi

def get_wiki_content(page_title):
  user_agent = "My Wikipedia Scraper (contact@example.com)"

  wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent)

  page = wiki_wiki.page(page_title)

  if page.exists():
      return page.text
  else:
      return None

In [ ]:
films_content = []
for film in film_list:
  films_content.append(get_wiki_content(film))

## Relation_Extraction


In [ ]:
!pip install openai
!pip install Neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Neo4j: filename=neo4j-5.18.0-py3-none-any.whl size=273862 sha256=31f72669d9146c31033384f70fd74062bb891c419a6277e866accb29c627e4f7
  Stored in directory: /root/.cache/pip/wheels/e7/e1/a0/dd7c19192f5383ff57d02a6c126cbfe4b7b2ae82f70c6994ce
Successfully built Neo4j


In [ ]:
import openai
import os
import re

os.environ['OPENAI_API_KEY'] = ""

from openai import OpenAI
client = OpenAI()

def extract_entity_relation(page_text):
  relations = "Ontology of triplets: (object; relation; object) with this relations \n (Movie title; Directed_by; Director)	\n (Movie title; Written_by; Writer1)	\n (Movie title; Written_by; Writer2) \n . \n . \n . \n (Movie title; Written_by; WriterN) \n (Movie title; Produced_by; Producer person)	\n (Movie title; Edited_by; Editor)	\n (Movie title; Production_company; company)	\n (Movie title; Distributed_by	; distributer)	\n (Movie title; Release_date; date)	\n (Movie title; Cast; Actor) \n"
  query_result = ""
  assistant = "Here is the text: \n" + page_text + "\n and Here is the triplets we want to extract for this texte: " + relations
  question = "Extract triplets from the text using the provided ontology. Replace the movie title, by the movie name as an entity. \n As response, only give the list of the triplets"



  messages = [ {"role": "system", "content": "You are an expert in Extracting Knowledge Graph relation"},
              {"role": "assistant", "content": assistant},
              {"role": "user", "content": question},
              ]

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  return response.choices[0].message.content


In [ ]:
films_entities_relations = []
for content in films_content:
  films_entities_relations.append(extract_entity_relation(content))

In [ ]:
import re
from neo4j import GraphDatabase

def extract_between_parentheses(line):
    match = re.search(r'\((.*?)\)', line)
    if match:
        return match.group(1)
    else:
        return None

def createneo4j_graph(string):
    # Initialiser une session Neo4j
    uri = ""
    user = ""
    password = ""
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        # Pour chaque ligne dans la chaîne de caractères
        for line in string.split('\n'):
            # Extraire ce qui se trouve entre parenthèses
            extracted = extract_between_parentheses(line)
            if extracted:
                movie, relation, value = extracted.split(';')
                type_relation = None
                movie = movie.strip()
                relation = relation.strip()
                value = value.strip()
                # Créer les nœuds et les relations dans la base de données Neo4j
                if relation in ['Directed_by', 'Written_by', 'Produced_by', 'Edited_by', 'Cast']:
                  type_relation = 'Person'
                  property_name = 'name'
                elif relation in ['Edited_by', 'Production_company', 'Distributed_by']:
                  type_relation = 'Company'
                  property_name = 'name'
                elif relation in ['Release_date']:
                  type_relation = 'Date'
                  property_name = 'date'
                session.run(
                    "MERGE (m:Movie {title: $movie})"
                    "MERGE (p:" + type_relation+ " {"+property_name+": $value})"
                    "MERGE (m)<-[:%s]-(p)" % relation,
                    movie=movie,
                    value=value)

In [ ]:
for entities_relations in films_entities_relations:
  createneo4j_graph(entities_relations)